In [1]:
# Import libraries and modules
import csv  # for reading and writing CSV files
import matplotlib.pyplot as plt  # for creating plots and charts
import numpy as np  # for numerical computing with Python
import pandas as pd  # for data manipulation and analysis
import pathlib  # for interacting with file paths in a cross-platform manner
import seaborn as sns  # for statistical data visualization
import tensorflow as tf  # for machine learning and deep learning
from keras.layers import Dense, Activation  # for building deep learning models in TensorFlow
from keras.models import Sequential  # for building deep learning models in TensorFlow
from keras.optimizers import Adam, RMSprop  # for building deep learning models in TensorFlow
from keras.callbacks import History
from sklearn.model_selection import train_test_split, GridSearchCV  # for model selection and evaluation
from sklearn.preprocessing import MinMaxScaler  # for preprocessing data
from tensorflow import keras  # for building deep learning models in TensorFlow
from tensorflow.keras import layers  # for building deep learning models in TensorFlow
from keras.wrappers.scikit_learn import KerasRegressor

# Print TensorFlow version
print(tf.__version__)

# Define column names for the CSV file
column_names = ['Open Time','Open','High','Low','Close',
                'Close Volume', 'Time', 'Quote asset volume',
               'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume']
               

# Read in the CSV file using pandas
raw_dataset = pd.read_csv("main.csv", names=column_names,
                      na_values = "?", comment='\t', dtype='float',
                      sep=",", skipinitialspace=True)

# Copy the data from the raw dataset to a new dataframe
dataset = raw_dataset.copy()

# Print the last few rows of the dataset
dataset.tail()

2023-01-07 22:11:42.945385: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-07 22:11:45.305851: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-07 22:11:45.306124: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-07 22:11:45.306142: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

2.11.0


,Open Time,Open,High,Low,Close,Close Volume,Time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
188312,1.620794e+12,57517.42,57526.28,57485.00,57485.07,42.575735,1.620794e+12,2.448258e+06,1195.0,15.319691,880913.090845
188313,1.620794e+12,57485.07,57496.42,57466.75,57481.49,34.205467,1.620794e+12,1.966194e+06,1096.0,15.971891,918058.816162
188314,1.620794e+12,57477.18,57509.99,57458.18,57470.00,30.211789,1.620794e+12,1.736514e+06,955.0,13.054229,750364.577319
188315,1.620794e+12,57470.00,57470.01,57400.00,57450.90,45.354728,1.620794e+12,2.605080e+06,1559.0,12.615628,724559.233035
188316,1.620794e+12,57450.89,57475.66,57435.51,57450.19,14.168318,1.620794e+12,8.140594e+05,730.0,7.247751,416412.022206


In [ ]:
dataset.pop("Open Time")
dataset.pop("Time")
# Randomly select 80% of the rows from the dataset and store them in a new dataframe
train_dataset = dataset.sample(frac=0.8,random_state=0)

# Remove the rows in the training dataset from the original dataset, leaving the remaining rows in a new dataframe
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
# Calculate statistical summary of the training dataset
train_stats = train_dataset.describe()

# Remove the "Close" column from the statistical summary
train_stats.pop("Close")

# Transpose the statistical summary so that it's in a more useful shape
train_stats = train_stats.transpose()

In [ ]:
# Remove the "Close" column from the training dataset and store it in a new dataframe
train_labels = train_dataset.pop('Close')

# Remove the "Close" column from the testing dataset and store it in a new dataframe
test_labels = test_dataset.pop('Close')

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_dataset)
normed_train_dataset = scaler.transform(train_dataset)
normed_test_dataset = scaler.transform(test_dataset)

In [ ]:
print(normed_train_dataset.shape)
print(train_labels.shape)

In [ ]:
print(train_labels)

In [ ]:
parameters = {
    'hidden_layer_sizes': [(128,), (32,), (32,32), (128,128)],
    'activation': ['tanh','relu','softmax'],
}

In [ ]:
gs = GridSearchCV(MLPRegressor(solver='adam'),param_grid=parameters,scoring="neg_mean_squared_error",n_jobs=-1,cv=5)
gs.fit(normed_train_dataset,train_labels)

In [ ]:
print("GridSearch:\n")
print("Best estimator:",gs.best_estimator_)
print("Best parameters:",gs.best_params_)
print("Score:",gs.best_score_)

In [2]:
def f(x):
    n_particles = x.shape[0]
    j = [objective(x[i])for i in range(n_particles)]
    return np.array(j)

def objective(neuronios, act_h):

    # Build the deep learning model with the fixed hyperparameters
    model = keras.Sequential([
        layers.Dense(neuronios, activation=act_h, input_shape=[8]),
        layers.Dense(neuronios, activation=act_h),
        layers.Dense(1,activation=act_h)
    ])

    # Compile the model
    optimizer = Adam(0.001)

    model.compile(loss='mse',optimizer=optimizer,metrics=['mae', 'mse'])
    # Train the model on the training data and labels
    model.fit(normed_train_dataset, train_labels, epochs=30)
    # Evaluate the model on the test data and labels
    score = model.evaluate(normed_test_dataset, test_labels, verbose=0)[0]
    return -score

In [3]:
# Get the best hyperparameters from grid search
best_params = gs.best_params_

# Set the fixed hyperparameters
neuronios = best_params['hidden_layer_sizes'][0]
act_h = best_params['activation']

# Create the PSO optimizer
options = {'c1': 0.5, 'c2': 0.3, 'w':1.5}
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=2, options=options)

# Optimize the model's parameters using the fixed hyperparameters
xopt = optimizer.optimize(lambda x: objective(neuronios, act_h), iters=5)

NameError: name 'gs' is not defined

In [ ]:
print(xopt)